# 0.0 Data preparation

# Preparation

Imports libraries

Loads data

In [ ]:
%matplotlib inline

print("0.0 data preparation")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.transforms as tr
import seaborn as sns
import re
import json
import datetime
import random

from random import randint
from ipywidgets import FloatProgress,IntProgress,IntText,Text,interact,interactive,IntSlider,FloatSlider
from IPython.display import display
from itertools import chain
from scipy.stats import ttest_ind

In [ ]:
pd.__version__

In [ ]:
np.__version__

In [ ]:
processRaw20190603 = not ('raw20190603' in globals())

### common variables

In [ ]:
dataFilesNamesStem = "2019-06-03_event_raw"
dataFolderPath = "../Data/"
csvSuffix = '.csv'

In [ ]:
questionCount = 12
firstLikertQuestionIndex = 0
lastLikertQuestionIndex = 11
questionArrayInt = [i+1 for i in range(questionCount)]
questionArrayStr = sorted(["Q" + "{0:0=2d}".format(i) for i in questionArrayInt])
questionArrayStr

In [ ]:
raw20190603Path = dataFolderPath + dataFilesNamesStem + csvSuffix

# Functions

## Loading

In [ ]:
#dateparseGForm = lambda x: pd.Timestamp(str(x).split(' GMT')[0], tz='Asia/Kolkata') #.tz_convert('utc')
raw20190603  = pd.read_csv(\
                            raw20190603Path,\
                            dtype=str,\
#                            parse_dates=['Timestamp'],\
#                            date_parser=dateparseGForm\
                         )

In [ ]:
raw20190603.columns

In [ ]:
#timestampQuestion = data20190603.columns[0]
#nameQuestion = data20190603.columns[1]
#emailQuestion = data20190603.columns[2]
gameQuestion = raw20190603.columns[0]
gameQuestion

In [ ]:
shortQuestions = pd.Series(index=questionArrayStr, data=raw20190603.columns[1:])
shortQuestions

In [ ]:
indexedQuestions = pd.Series(index=range(1,13), data=raw20190603.columns[1:])
indexedQuestions

In [ ]:
indexedLikertQuestions = indexedQuestions[firstLikertQuestionIndex:lastLikertQuestionIndex]
indexedLikertQuestions

In [ ]:
shortLikertQuestions = shortQuestions[firstLikertQuestionIndex:lastLikertQuestionIndex]
shortLikertQuestions

In [ ]:
games = raw20190603[gameQuestion].unique()
games

In [ ]:
gameDrBugTitle = 'Dr Bug: Microbe Mayhem'
gameSuperbugsTitle = 'Superbugs: the Game'
gameFungalTitle = 'Fungal Invasion'

In [ ]:
shortGameNames = pd.Series(index=games, data=['Dr Bug', 'Superbugs', 'Fungal Invasion'])
shortGameNames

In [ ]:
def getShortGameTitle(longGameName):
    return shortGameNames.get(longGameName, "unknown")

In [ ]:
data20190603 = raw20190603.copy()
for question in indexedLikertQuestions:
    for respondent in data20190603.index:
        try:
            data20190603.loc[respondent, question] = int(data20190603.loc[respondent, question])
        except:
            data20190603.loc[respondent, question] = np.nan

In [ ]:
positiveQuestions = indexedLikertQuestions.copy()
positiveQuestions = positiveQuestions[positiveQuestions.index % 2 == 1]
negativeQuestions = indexedLikertQuestions.copy()
negativeQuestions = negativeQuestions[negativeQuestions.index % 2 == 0]

In [ ]:
# transforms the agreement scores (1-5 Likert scale) into a 0-4 mark with 0 = bad and 4 = great
data20190603SUSNormalized = data20190603.copy()
for respondant in data20190603SUSNormalized.index:
    for question in indexedLikertQuestions:
        answer = data20190603SUSNormalized.loc[respondant, question]
        if pd.notna(answer):
            if question in negativeQuestions.values:            
                data20190603SUSNormalized.loc[respondant, question] = answer-1
            else:
                data20190603SUSNormalized.loc[respondant, question] = 5-answer